In [15]:
import pandas as pd 
data = pd.read_csv('202101-202106(1).csv',encoding='utf-8')
data['上线月份'] = pd.to_datetime(data['上线月份'], format='%Y%m').dt.strftime('%Y-%m')
data.head()

,零件号,仓库,相似ID,新旧,上线月份,1,2,3,4,5,6
0,56D601025C 8Z8,3001,D60102-3001,old,2016-04,31,22.0,20.0,22.0,27.0,13.0
1,5GD853671E 9B9,8012,D85367-8012,old,2018-11,1,2.0,1.0,6.0,6.0,5.0
2,18D839698F,8015,D83969-8015,old,2016-04,5,7.0,4.0,5.0,9.0,8.0
3,ZVI000001 ROT,1000-3,I00000-1000-3,new,2021-02,46,38.0,163.0,123.0,90.0,49.0
4,3CC821112,3001,C82111-3001,new,2021-01,5,3.0,2.0,NaN,2.0,14.0


In [16]:
real_demods_info = data[data['新旧']=='new']
real_demods_info

,零件号,仓库,相似ID,新旧,上线月份,1,2,3,4,5,6
3,ZVI000001 ROT,1000-3,I00000-1000-3,new,2021-02,46,38.0,163.0,123.0,90.0,49.0
4,3CC821112,3001,C82111-3001,new,2021-01,5,3.0,2.0,NaN,2.0,14.0
5,LVM765035Z2,5003,M76503-5003,new,2021-06,6,NaN,NaN,NaN,NaN,NaN
16,PHN02011101,8023,N02011-8023,new,2021-03,1200,492.0,256.0,837.0,103.0,400.0
26,11D821022,8022,D82102-8022,new,2021-03,13,NaN,9.0,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
52674,3GB853689C SGU,8015,B85368-8015,new,2021-03,1,NaN,NaN,NaN,NaN,NaN
52676,3ED807724B,8013,D80772-8013,new,2021-03,1,1.0,NaN,NaN,NaN,NaN
52681,18A721039,1000-1,A72103-1000-1,new,2021-04,1,NaN,1.0,2.0,NaN,NaN
52691,ZCX01911303,5002,X01911-5002,new,2021-04,6,NaN,NaN,NaN,NaN,NaN


In [17]:
pred_svg_info = pd.read_csv('svg预测(2).csv',encoding='utf-8')
pred_svg_info.head()

,零件号,仓库,上线月份,1,2,3,4,5,6
0,11D807185B D7X,8012,202105,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,5NH807221 GRU,8012,202101,1.0,0.333333,0.666667,0.4,0.266667,0.190476
2,3CG858819,1000-1,202104,NaN,0.333333,0.166667,0.1,0.066667,0.047619
3,5NG831055K,1000-1,202104,NaN,NaN,NaN,NaN,NaN,NaN
4,34D853651H RYP,3002,202105,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
output = pd.merge(real_demods_info,pred_svg_info,on =['零件号','仓库'],how ='left')
for i in ['1_y','2_y','3_y','4_y','5_y','6_y']:
    output[i] = output[i].fillna(1)

In [22]:
import matplotlib.pyplot as plt
import seaborn as sns
def compare_two_col(r,p):
    new_df = output[[r,p]].dropna(subset=r)
    new_df['mape'] = abs(new_df[p]-new_df[r])/new_df[r]
    
    bins = [0, 0.10, 0.30, 0.50, 1.00, 100000]
# 使用cut函数将数据分成不同区间
    new_df['mape_rate'] = pd.cut(new_df['mape'], bins=bins, labels=['0-10%', '10%-30%', '30%-50%', '50%-100%', '>100%'],right =False)
    new_df['mape_rate'].replace('NaN','0',inplace =True)
# 计算每个区间中的数据数量
    print(new_df['mape'].mean())
    print(new_df['mape_rate'].value_counts(normalize=True))
    
    
print('2月份')
compare_two_col('2_x','1_y')
print('3月份')

compare_two_col('3_x','2_y')
print('4月份')

compare_two_col('4_x','3_y')
print('5月份')

compare_two_col('5_x','4_y')
print('6月份')

compare_two_col('6_x','5_y')




2月份
1.3848800261851844
50%-100%    0.4004
0-10%       0.2716
>100%       0.2120
30%-50%     0.0648
10%-30%     0.0512
Name: mape_rate, dtype: float64
3月份
1.6721681854619201
50%-100%    0.447687
>100%       0.234520
0-10%       0.134164
30%-50%     0.096085
10%-30%     0.087544
Name: mape_rate, dtype: float64
4月份
0.8594983264483963
50%-100%    0.407895
>100%       0.194879
10%-30%     0.145804
30%-50%     0.130868
0-10%       0.120555
Name: mape_rate, dtype: float64
5月份
0.7656841296125895
50%-100%    0.434100
30%-50%     0.162468
>100%       0.152610
0-10%       0.129609
10%-30%     0.121212
Name: mape_rate, dtype: float64
6月份
0.8678890565051581
50%-100%    0.391474
30%-50%     0.163055
10%-30%     0.154529
>100%       0.154174
0-10%       0.136767
Name: mape_rate, dtype: float64
